# Parsing APIs Example

## Intro

Now we will take a look on a real data. When you parse data from web you will often meet API based web-pages. 

For example [zalando.fr](https://www.zalando.fr/accueil-homme/) is API based web-page. 

In this guided lab you will learn how to obtain the links from webpages and extract the data. Read through this doc, execute the cells in order and make sure you understand the explanations. 

*Note: This guided lab uses Google Chrome. Other browsers like Safari and Firefox have similar tools for developers but they work differently. To save your time in following this lab, it is strongly recommended that you install and use Google Chrome.*

## Obtaining the link

Zalando is discount e-store where you can buy clothes and accesories with discount. When we go to the web-page, we can choose different sections. First the general process will be shown using [Children section](https://www.zalando.fr/accueil-enfant/) as example.

Here we will parse data about promotions only. Therefore, final output will be the DataFrame with all the goods under discount.

[![Image from Gyazo](https://i.gyazo.com/fa4874d8e81c7570273bbfb853d66308.png)](https://gyazo.com/fa4874d8e81c7570273bbfb853d66308)


We go to Promos page. Right click of mouse shows us a list of actions possible, from which we select Inspect.

<img src='https://i.gyazo.com/bccbd11d69c9040dc98758d443e32052.png' width="400">


You will see the menu dropdown on the right side or on the bottom of the window. There you should click on Network:


[![Image from Gyazo](https://i.gyazo.com/f7e0db81cbfee67694183d1a7640bf81.png)](https://gyazo.com/f7e0db81cbfee67694183d1a7640bf81)

Right after the developer part will change showing the files behind the page. In order to obtain only useful files we select the following settings:
1. Preserve Log
2. Select XHR files.

[![Image from Gyazo](https://i.gyazo.com/9a899d4441d9d93e795f79747f1e47d5.png)](https://gyazo.com/9a899d4441d9d93e795f79747f1e47d5)

In order to obtain some files we need to scrool down and go forward to second page. 

[![Image from Gyazo](https://i.gyazo.com/0956eb3d5125075a236c9a439c7749c7.png)](https://gyazo.com/0956eb3d5125075a236c9a439c7749c7)

In the Network panel you can see the following files being uploaded. All the data on the web-page is uploaded from the json file, which is one of the following. It is important to understand which file contains what kind of information. 

<a href="https://gyazo.com/cf97a655869f0b22df0ada1cb2a41c3c"><img src="https://i.gyazo.com/cf97a655869f0b22df0ada1cb2a41c3c.png" alt="Image from Gyazo" width="724.8"/></a>

When you find what kind of information you need for the data to be uploaded you just test it. Here we need the article... file:

<a href="https://gyazo.com/78b35bf492994b3f35c0564a21da202a"><img src="https://i.gyazo.com/78b35bf492994b3f35c0564a21da202a.png" alt="Image from Gyazo" width="727.2"/></a>

When we test the link in Chrome inkognito mode we obtain the proper json file:


<a href="https://gyazo.com/b60453fa98454fa29771c731a5174443"><img src="https://i.gyazo.com/b60453fa98454fa29771c731a5174443.png" alt="Image from Gyazo" width="1530.4"/></a>

In order to change the objects in the json file (kind of pagination), you need to change the offset (the number of the first element on the page). in fact, if you take a look on the link, it is easy to unerstand the structure of the link.

# Reading the data

Now the party rocks! When we know how can we obtain the data, it is not a problem to obtain the whole database with all the data from the web-page.
In this lab you will collect your database of Zalando products. You select which goods you want to track. You can define as many filters to your data as you want. Just make sure that the data represents the filters.




In [22]:
import json
import requests
import pandas as pd
from pandas.io.json import json_normalize


In [23]:
requests.get('http://www.google.com')

<Response [200]>

In [24]:
# Paste the url you obtained for your data
url='https://www.zalando.fr/api/catalog/articles?categories=promo-enfant&limit=84&offset=84&sort=sale'


In [28]:
#requests.get(url)

headers = requests.utils.default_headers()
headers['User-Agent'] = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36'
response = requests.get(url, headers=headers)
response

<Response [200]>

#### Collect first 84 object of the of the data (1st page)

Your output should be a Pandas DataFrame of goods. Each row should contain only text or numbers, having *family_articles, flags, media* and *sizes* remaining lists (they are exceptions).

In [30]:
results = response.json()
#results

In [31]:
flattened_data = json_normalize(results)

In [32]:
flattened_data1 = json_normalize(flattened_data.articles[0])


In [34]:
#flattened_data1

#### Collect all the objects from selected filters. Total number of pages can be found in the same json. Use *sku* column as index.

Your output should be a Pandas DataFrame of goods. Each row should contain only text or numbers, having family_articles, flags, media and sizes remaining lists (they are exceptions).

In [36]:
# Get the total number of pages
total_pages=results['pagination']['page_count']

# Your code
df=pd.DataFrame()
for i in range(total_pages):
    k=84*i
    url=f'https://www.zalando.fr/api/catalog/articles?categories=promo-enfant&limit=84&offset={k}&sort=sale'
    headers = requests.utils.default_headers()
    headers['User-Agent'] = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36'
    response = requests.get(url, headers=headers)
    results = response.json()
    flattened_data = json_normalize(results)
    flattened_data1 = json_normalize(flattened_data.articles[0])
    flattened_data1=flattened_data1.set_index('sku')
    df = df.append(flattened_data1)

df

/Users/julieG/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


,amount,brand_name,family_articles,flags,is_premium,media,name,price.has_different_original_prices,price.has_different_prices,price.has_different_promotional_prices,price.has_discount_on_selected_sizes_only,price.original,price.promotional,product_group,sizes,url_key
sku,,,,,,,,,,,,,,,,
NI114F00V-C11,NaN,Nike Sportswear,[],"[{'key': 'campaign', 'value': '-15% EXTRA', 't...",False,[{'path': 'NI/11/4F/00/VC/11/NI114F00V-C11@4.j...,REACT PRESTO - Chaussures premiers pas - wolf ...,False,True,True,False,"59,95 €","24,00 €",shoe,"[21, 22, 23.5, 25, 26, 27]",nike-sportswear-nike-react-presto-baskets-bass...
N1243E0J0-Q11,NaN,Nike Performance,[],"[{'key': 'campaign', 'value': '-15% EXTRA', 't...",False,[{'path': 'N1/24/3E/0J/0Q/11/N1243E0J0-Q11@4.j...,AIR PANT - Pantalon de survêtement - black,False,True,True,False,"44,95 €","13,49 €",clothing,"[6-8a, 8-10a, 10-12a, 12-13a, 13-15a]",nike-performance-air-pant-pantalon-de-survetem...
N1243A0XZ-C11,82 g,Nike Performance,[],"[{'key': 'campaign', 'value': '-15% EXTRA', 't...",False,[{'path': 'N1/24/3A/0X/ZC/11/N1243A0XZ-C11@9.j...,FLEX RUNNER - Chaussures de running neutres - ...,False,False,False,False,"29,95 €","18,00 €",shoe,"[17, 18.5, 19.5, 21, 22, 23.5, 25, 26]",nike-performance-flex-runner-chaussures-de-run...
N1243G01D-N11,NaN,Nike Performance,[],"[{'key': 'campaign', 'value': '-15% EXTRA', 't...",False,[{'path': 'N1/24/3G/01/DN/11/N1243G01D-N11@4.j...,FULL ZIP - veste en sweat zippée - cargo khaki...,False,False,False,False,"39,95 €","16,00 €",clothing,"[8-10a, 10-12a, 12-13a, 13-15a]",nike-performance-ya76-veste-en-sweat-n1243g01d...
NI114D0A1-C12,NaN,Nike Sportswear,[],"[{'key': 'campaign', 'value': '-15% EXTRA', 't...",False,[{'path': 'NI/11/4D/0A/1C/12/NI114D0A1-C12@7.j...,REACT PRESTO - Baskets basses - wolf grey/white,False,True,True,False,"94,95 €","38,00 €",shoe,"[35.5, 36, 36.5, 37.5, 38, 38.5, 39, 40]",nike-sportswear-react-presto-baskets-basses-ni...
F5723K01F-G11,NaN,Friboo,[],"[{'key': 'campaign', 'value': '-15% EXTRA', 't...",False,[{'path': 'F5/72/3K/01/FG/11/F5723K01F-G11@3.j...,Sweat à capuche - chrysanthemium,False,True,True,False,"17,95 €","5,40 €",clothing,"[3-4a, 5-6a, 7-8a, 9-10a, 11-12a]",friboo-sweatshirt-chrysanthemium-f5723k01f-g11
R2724G00L-N11,NaN,Redskins,[],"[{'key': 'campaign', 'value': '-15% EXTRA', 't...",False,[{'path': 'R2/72/4G/00/LN/11/R2724G00L-N11@4.j...,TENNYSON - T-shirt imprimé - kaki light,False,False,False,False,"22,95 €","8,00 €",clothing,"[10a, 14a, 16a]",redskins-tennyson-t-shirt-imprime-kaki-light-r...
RE016D03K-A11,NaN,Reebok Classic,[],"[{'key': 'campaign', 'value': '-15% EXTRA', 't...",False,[{'path': 'RE/01/6D/03/KA/11/RE016D03K-A11@2.1...,WORKOUT PLUS - Baskets basses - white,False,False,False,False,"59,95 €","21,00 €",shoe,"[34.5, 35, 36, 36.5, 37, 38]",reebok-classic-workout-plus-baskets-basses-whi...
N1243G09R-Q11,NaN,Nike Performance,[],"[{'key': 'campaign', 'value': '-15% EXTRA', 't...",False,[{'path': 'N1/24/3G/09/RQ/11/N1243G09R-Q11@9.j...,HOODIE STUDIO - veste en sweat zippée - black/...,False,False,False,False,"44,95 €","18,00 €",clothing,"[6-8a, 8-10a, 10-12a, 12-13a, 13-15a]",nike-performance-hoodie-studio-veste-en-sweat-...


#### Display the trending brand in DataFrame

In [37]:
df.brand_name.value_counts().index[0]

'Friboo'

#### Display the brand with maximal total discount (sum of discounts on all goods)

In [38]:
#Our data is still text. Convert prices into numbers:
df['price.original']=df['price.original'].str.extract('(\d*,\d*)')
df['price.promotional']=df['price.promotional'].str.extract('(\d*,\d*)')

df['price.original'] = [x.replace(',', '.') for x in df['price.original']]
df['price.promotional'] = [x.replace(',', '.') for x in df['price.promotional']]

In [39]:
df['discount_amount']=df['price.original'].astype(float)-df['price.promotional'].astype(float)
df1=df.copy()

In [40]:
total_disc=df1.groupby(['brand_name']).sum().discount_amount

In [41]:
total_disc.sort_values(ascending=False).index[0]

'Friboo'

#### Display the brands without discount at all

In [42]:
total_disc[total_disc==0]

brand_name
Teva    0.0
Name: discount_amount, dtype: float64